# 📸 FOTEI: Cloud Logic Hub (Google Colab)

このノートブックは、ローカルで整理・圧縮されたメディアをクラウド（Google Photos / Drive）へ同期し、AI解析するための実行環境です。

## 🛠 Setup
1. Google Driveをマウントします。
2. 認証ファイル (`credentials.json`) をアップロードしてください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 必要なライブラリのインストール
%pip install google-auth-oauthlib google-api-python-client requests piexif

## 🧬 Common Logic
認証やAPIクライアントの初期化など、共通の関数を定義します。

In [ ]:
import os
import json
import pickle
import requests
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

def get_service(api_name, api_version, account_name, scopes):
    """汎用的なGoogle APIサービスの取得プログラム"""
    creds = None
    token_file = f'token_{api_name}_{account_name}.pickle'
    
    if os.path.exists(token_file):
        with open(token_file, 'rb') as token:
            creds = pickle.load(token)
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists('credentials.json'):
                print("Error: 'credentials.json' not found. Please upload it.")
                return None
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', scopes)
            # Colab環境ではコンソールモードで認証を行う場合が多いが、run_local_serverでも動作することがある
            creds = flow.run_local_server(port=0)
        with open(token_file, 'wb') as token:
            pickle.dump(creds, token)

    return build(api_name, api_version, credentials=creds, static_discovery=False if api_name == 'photoslibrary' else True)

## Phase 3: Photo Cloud Sync (Google Photos)
ローカルで圧縮した写真をGoogle Photosへバックアップします。

In [ ]:
def upload_to_photos(service, file_path):
    filename = os.path.basename(file_path)
    with open(file_path, 'rb') as f:
        data = f.read()

    token = service._http.credentials.token
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'application/octet-stream',
        'X-Goog-Upload-File-Name': filename,
        'X-Goog-Upload-Protocol': 'raw',
    }

    response = requests.post('https://photoslibrary.googleapis.com/v1/uploads', headers=headers, data=data)
    if response.status_code == 200:
        upload_token = response.content.decode('utf-8')
        service.mediaItems().batchCreate(body={
            'newMediaItems': [{'simpleMediaItem': {'uploadToken': upload_token}}]
        }).execute()
        return True
    return False

# 実行例
# photo_service = get_service('photoslibrary', 'v1', 'user1', ['https://www.googleapis.com/auth/photoslibrary.appendonly'])
# upload_to_photos(photo_service, '/content/drive/MyDrive/FOTEI/test.jpg')

## Phase 4: Screenshot Semantic Cloud (Drive/AI)
スクショをDriveへ送り、AI (Gemini) で解析します。

In [ ]:
def upload_to_drive(service, file_path, folder_id=None):
    file_metadata = {'name': os.path.basename(file_path)}
    if folder_id: file_metadata['parents'] = [folder_id]
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

# 実行例
# drive_service = get_service('drive', 'v3', 'user1', ['https://www.googleapis.com/auth/drive.file'])
# upload_to_drive(drive_service, '/content/drive/MyDrive/FOTEI/screenshot.png')